In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [62]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

X = df_train.drop(['target', 'id'], axis=1)
y = df_train['target']

X_test = df_test.drop('id', axis=1)

In [55]:
X.nunique()

f0       26
f1        5
f2     1499
f3     1206
f4     7023
f5        2
f6        3
f7        3
f8        6
f9      190
f10       4
f11     190
f12       6
f13     222
f14       2
f15      26
f16      12
f17       6
f18     222
f19       7
f20    2167
f21       3
f22       2
f23       6
f24       2
f25     104
f26     222
f27      15
f28       2
dtype: int64

In [56]:
for col in X:
    print(col)
    print(sorted(X[col].dropna().unique()))

f0
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
f1
['AU', 'CN', 'DE', 'IT', 'JP']
f2
['001fa96ad', '003b53013', '003d0e1d2', '007c05fbd', '0092cddd7', '00a1ea17b', '00bbcfe3e', '00cd56f7e', '0137900c0', '0147cbb6a', '015a2ae65', '0160d64b1', '0166373e7', '0173b7eae', '017d385e1', '020b0e75a', '02280236e', '02376296a', '0264452a0', '0266238ef', '02b92ef0f', '031e5a41f', '035095152', '035134eb8', '0356009d6', '03a6514a0', '03a875273', '03aa3051d', '04130311d', '04174386c', '04a2f06a3', '04c293b6e', '0558225bd', '05b7e6255', '05c0bb5eb', '05c6d2268', '05cb6c88e', '05cd24314', '065ad737e', '06776c29a', '06abf8307', '06cd379f8', '0702bd290', '0730804d2', '0734e1e19', '0739a61d7', '073e9dcff', '074c2bfb3', '0755f5a0f', '0799d590c', '083cce223', '084ae7064', '0850be70d', '0872167c6', '08c2cdd34', '08cb1dd36', '08d3ff40a', '08dd55d0f', '08dd65794', '091b72312', '093767d8a', '098d9fef5', '09a47e43b', '09e90a52

In [57]:
X['f4'].sort_values()

48083    6.5218
36924    6.5222
11173    6.5321
35655    6.5524
37807    6.5561
          ...  
49831       NaN
49837       NaN
49876       NaN
49901       NaN
49965       NaN
Name: f4, Length: 50000, dtype: float64

In [58]:
binary_cols = ['f14', 'f22', 'f28']
orinal_cols = ['f0', 'f5', 'f6', 'f8', 'f9', 'f10', 'f11', 'f15', 'f16', 'f17', 'f19', 'f21', 'f23', 'f25', 'f27'] # f10 is ordinal if converted from binary to dec
nominal_cols = ['f1', 'f2', 'f3', 'f4', 'f7', 'f12', 'f13', 'f18', 'f20', 'f24', 'f26']

high_cardinality = [c for c in nominal_cols if len(X[c].unique()) > 16]
low_cardinality = [c for c in nominal_cols if len(X[c].unique()) <= 16]

In [63]:
X['f9_1'] = X['f9'].apply(lambda x: x[0] if type(x) == str else np.nan)
X['f9_2'] = X['f9'].apply(lambda x: x[1] if type(x) == str else np.nan)
X_test['f9_1'] = X_test['f9'].apply(lambda x: x[0] if type(x) == str else np.nan)
X_test['f9_2'] = X_test['f9'].apply(lambda x: x[1] if type(x) == str else np.nan)

X['f11_1'] = X['f11'].apply(lambda x: x[0] if type(x) == str else np.nan)
X['f11_2'] = X['f11'].apply(lambda x: x[1] if type(x) == str else np.nan)
X_test['f11_1'] = X_test['f11'].apply(lambda x: x[0] if type(x) == str else np.nan)
X_test['f11_2'] = X_test['f11'].apply(lambda x: x[1] if type(x) == str else np.nan)

In [64]:
def binaryToDecimal(binary):
    try:
        binary = int(binary)
    except:
        return np.nan

    binary1 = binary
    decimal, i, n = 0, 0, 0
    while(binary != 0):
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)
        binary = binary//10
        i += 1
    return decimal
    for col in binary_columns:
        if col not in list(df.columns):
            continue
            
X['f10'] = X['f10'].apply(lambda x: binaryToDecimal(x))

In [66]:
from sklearn.compose import make_column_transformer
from imblearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

ordinal_transformer = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder()
)

encoders = [OrdinalEncoder() for _ in range(orinal_cols)]

for col, column in enumerate(orinal_cols):
    unique_values = pd.Series(X[column].append(X_test[column]).unique())
    unique_values = unique_values[unique_values.notnull()]
    encoders[col].fit(unique_values)
    X.loc[X[column].notnull(), column] = label_encoders[col].transform(X.loc[X[column].notnull(), column])
    X_test.loc[X_test[column].notnull(), column] = label_encoders[col].transform(X_test.loc[X_test[column].notnull(), column])

ValueError: DataFrame constructor not properly called!

In [44]:
X.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [29]:
from sklearn.preprocessing import LabelEncoder

label_encoders = [LabelEncoder() for _ in range(X.shape[1])]

for col, column in enumerate(X.columns):
    unique_values = pd.Series(X[column].append(X_test[column]).unique())
    unique_values = unique_values[unique_values.notnull()]
    label_encoders[col].fit(unique_values)
    X.loc[X[column].notnull(), column] = label_encoders[col].transform(X.loc[X[column].notnull(), column])
    X_test.loc[X_test[column].notnull(), column] = label_encoders[col].transform(X_test.loc[X_test[column].notnull(), column])
    
X.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f9_1,f9_2,f11_1,f11_2
0,8,0,141,908,2643.0,1,0.0,2,3.0,54,1.0,54,0,7,1,8,8.0,3.0,26,2.0,1658,2.0,0.0,1,0,50.0,26,12,1.0,15,18,15,18
1,13,NaN,230,276,1813.0,0,1.0,1,5.0,14,2.0,14,5,76,1,13,11.0,4.0,200,0.0,1007,2.0,1.0,5,0,9.0,200,10,1.0,3,19,3,19
2,15,4,618,678,1476.0,0,0.0,2,4.0,175,0.0,175,2,115,1,15,4.0,1.0,88,1.0,1678,2.0,0.0,2,1,15.0,88,2,0.0,46,41,46,41
3,10,3,932,1045,3017.0,0,0.0,2,4.0,7,1.0,7,0,16,0,10,1.0,3.0,170,2.0,1843,1.0,0.0,3,0,62.0,170,8,1.0,1,47,1,47
4,7,0,812,1149,3953.0,0,1.0,1,4.0,126,0.0,126,3,46,1,7,7.0,5.0,5,5.0,NaN,1.0,1.0,0,1,25.0,5,7,1.0,35,1,35,1
